# Image Transformer
1. Image 투명화 
2. 투명 처리된 Image 저장
3. Image 정보(Category, Boundary)를 csv에 저장

In [1]:
import Img_Maker
# 이미지 파일 경로 입력 (메모리 최소 800MB 필요)

# 1 : 이미지 저장
# 0 : 패스
# 2 : 종료

# JSON File Path 설정
FILE_PATH = r'D:\MODEL_DATA\AA\4.json'
Img_Maker.check_img(FILE_PATH = FILE_PATH, save_pos = 0)

마지막 Save Point 37


# 참고자료
https://studyforus.com/innisfree/594134

# 남은 작업
## 학습 데이터 생성기
1. 매장 이미지(1024x1024)에 투명화한 상품 합치기
2. 매장 이미지 별 BOX 정보 COCO 파일 저장

In [3]:
import pandas as pd

In [4]:
pd.read_csv(r'C:\Users\svsta\EasyFree\IMAGE\coco_file.csv')

,image_name,category,product,x,y,w,h
0,Train_Image/abc,6510158,1000048751091,111,111,119,119


In [2]:
# 이미지 합치기
from PIL import Image

In [135]:
background = Image.open("emart.png")
item = Image.open('Train_Image/abc.png')

In [67]:
import numpy as np

In [ ]:
np.random.randint(background.size[0]) - item.size[0]
np.random.randint(background.size[1]) - item.size[1]

In [160]:
background.paste(item, (np.random.randint(item.size[0]//2, background.size[0] - item.size[0]) ,np.random.randint(item.size[1]//2, background.size[1] - item.size[1])), item)
#.resize((112,112))

# Random Position 이지만 겹치지 않아야 함
# 박스정보 X,Y,W,H로 위치 미세조정
# cocofile에 넣을 값 = 위치 pos에 center x, center y 를 더하면 됨

In [2]:
background

NameError: name 'background' is not defined